In [1]:
import pandas as pd
import numpy as np
import numpy_indexed as npi
from itertools import combinations, product, permutations
import itertools
from functools import reduce
from collections.abc import Iterable
import math
# use for vertify
from mlxtend.frequent_patterns import apriori, association_rules


In [2]:
real_dataset_path = "/home/sokhorn/sokhorn/dataSet/data/Online Retail.csv"
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv'

sample_dataset = pd.read_csv(
    testing_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity'
    ])
sample_dataset


,InvoiceNo,StockCode,Quantity
0,536365,85123A,6
1,536365,71053,6
2,536365,84406B,8
3,536365,84029G,6
4,536365,84029E,6
5,536365,22752,2
6,536365,21730,6
7,536366,22633,6
8,536366,22632,6
9,536367,84879,32


In [3]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
item_sets


StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,6.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,6.0,6.0,8.0,0.0,6.0
536366,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536368,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536369,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
en_code = lambda x : 1 if x > 0 else 0 
item_sets = item_sets.applymap(en_code)
item_sets


StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


# sort column 

In [75]:
item_sets = item_sets.reindex(sorted(item_sets.columns), axis=1)
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
k_itemsts = np.array(item_sets.columns)
k_itemsts

array(['21730', '22632', '22633', '22745', '22748', '22749', '22752',
       '71053', '84029E', '84029G', '84406B', '84879', '85123A'],
      dtype=object)

In [8]:
item_tranctions = item_sets.values  
item_tranctions

array([[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [9]:
item_setr_after_reduct, rc =  npi.count(item_tranctions) # find RC count 

In [11]:
class myarray(np.ndarray):
    def __new__(cls, *args, **kwargs):
        return np.array(*args, **kwargs).view(myarray)

    def index(self, value):
        return np.where(self == value)


In [12]:
index = lambda x : np.where(k_itemsts == x)

In [ ]:
item_tranctions = np.array(item_tranctions,)
item_tranctions

In [38]:
unq,tags = np.unique(k_itemsts, return_inverse=True)
dict(zip(range(len(unq)),unq))
tags

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [41]:
np.sort(k_itemsts)

(13,)

In [69]:
np.arange(len([0, 4, 1, 3, 2]))

array([0, 1, 2, 3, 4])

In [76]:
def gen():
    for i in range(5):
        yield i, i + 1


In [77]:
for i, j in gen():
    print(i, j)

0 1
1 2
2 3
3 4
4 5
